In [2]:

%pip install evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print GPU status
if torch.cuda.is_available():
    print(f"✓ GPU is available and will be used: {torch.cuda.get_device_name(0)}")
    print(f"  Number of GPUs: {torch.cuda.device_count()}")
else:
    print("✗ No GPU detected. Training will use CPU.")
    
print(f"Device: {device}")

imdb = load_dataset("imdb")

train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(10000))])
test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(10000))])

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True, padding=True, max_length=256)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

✓ GPU is available and will be used: Tesla T4
  Number of GPUs: 1
Device: cuda


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


TrainOutput(global_step=939, training_loss=0.2234922171400759, metrics={'train_runtime': 998.9253, 'train_samples_per_second': 30.032, 'train_steps_per_second': 0.94, 'total_flos': 1987010979840000.0, 'train_loss': 0.2234922171400759, 'epoch': 3.0})

In [3]:
from transformers import TextClassificationPipeline
sentiment = TextClassificationPipeline(model=model, tokenizer=tokenizer)
sentiment([
    "Absolutely terrible waste of time",
    "The acting was phenomenal and emotionally powerful",
    "It was okay, nothing special",
    "Worst film of the decade",
    "I would definitely recommend this to my friends",
    "I would not recommend this to my worst enemy"
])


[{'label': 'LABEL_0', 'score': 0.9407925605773926},
 {'label': 'LABEL_1', 'score': 0.9613524079322815},
 {'label': 'LABEL_0', 'score': 0.7723464965820312},
 {'label': 'LABEL_0', 'score': 0.9107998013496399},
 {'label': 'LABEL_1', 'score': 0.8486443161964417},
 {'label': 'LABEL_0', 'score': 0.7440579533576965}]